## Preprocessing

In [1]:
# Import our dependencies
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.activations import sigmoid, relu, tanh
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt

Using TensorFlow backend


In [2]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df= application_df.drop(['EIN','NAME'], axis=1)

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
app_counts= application_df['APPLICATION_TYPE'].value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
app_cutoff= 1000
application_types_to_replace=app_counts[app_counts<app_cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
cls_counts= application_df['CLASSIFICATION'].value_counts()
cls_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
multiple_cls_counts= cls_counts[cls_counts>1]
multiple_cls_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cls_cutoff= 1500
classifications_to_replace= cls_counts[cls_counts<cls_cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
columns = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df = pd.get_dummies(application_df, columns=columns)

In [11]:
# Split our preprocessed data into our features and target arrays
y= application_df['IS_SUCCESSFUL']
X= application_df.drop(['IS_SUCCESSFUL'], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train,X_test, y_train,y_test= train_test_split(X,y)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
def model_builder(hp):
    model = Sequential()

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=1, max_value=1000, step=100)
    hp_layer_2 = hp.Int('layer_2', min_value=1, max_value=1000, step=100)
    hp_layer_3 = hp.Int('layer_3', min_value=1, max_value=1000, step=100) 
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(tf.keras.layers.Input(shape=(X_train.shape[1],))) 
    model.add(tf.keras.layers.Dense(units=hp_layer_1, activation=hp_activation))
    model.add(tf.keras.layers.Dense(units=hp_layer_2, activation=hp_activation))
    model.add(tf.keras.layers.Dense(units=hp_layer_3, activation=hp_activation))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=Adam(learning_rate=hp_learning_rate),
        loss=BinaryCrossentropy(),
        metrics=['accuracy']
    )
  
    return model

In [14]:
# Create a Keras Tuner Hyperband tuner to search for the best hyperparameters
tuner = kt.Hyperband(
    model_builder,          # The model builder function
    objective='accuracy',   # The optimization objective is to maximize accuracy
    max_epochs=10,          # The maximum number of epochs to train each model
    factor=3,               # Reduction factor for the number of models in each bracket
    directory='Models',     # Directory to store tuning results
    project_name='alphabet_soup_charity'  # Name of the project
)

Reloading Tuner from Models/alphabet_soup_charity/tuner0.json


In [15]:
# Define an early stopping callback
stop_early = EarlyStopping(
    monitor='loss',   # Monitor the loss during training
    patience=3         # Number of epochs with no improvement to wait before stopping
)

In [16]:
# Search for the best hyperparameters using Keras Tuner
tuner.search(
    X_train_scaled,     # Training features
    y_train,            # Training labels
    epochs= 50,         # Number of training epochs
    callbacks=[stop_early]  # Early stopping callback to stop training if needed
)

In [17]:
# Retrieve the best hyperparameters found by Keras Tuner
best_hps = tuner.get_best_hyperparameters(num_trials=1)

In [18]:
# Print the best hyperparameters
for i, hyperparameters in enumerate(best_hps):
    print(f"Top-{i + 1} Hyperparameters:")
    print(f"Activation Function: {hyperparameters['activation']}")
    print(f"Number of Units in Layer 1: {hyperparameters['layer_1']}")
    print(f"Number of Units in Layer 2: {hyperparameters['layer_2']}")
    print(f"Number of Units in Layer 3: {hyperparameters['layer_3']}")
    print(f"Learning Rate: {hyperparameters['learning_rate']}")
    print("Number of Hidden Layers: 3")

Top-1 Hyperparameters:
Activation Function: relu
Number of Units in Layer 1: 101
Number of Units in Layer 2: 401
Number of Units in Layer 3: 901
Learning Rate: 0.001
Number of Hidden Layers: 3


In [19]:
# Build the neural network model using the best hyperparameters
model = tuner.hypermodel.build(best_hps[0])

# Train the model on the training data
model.fit(X_train_scaled, y_train, epochs=80, callbacks=[stop_early])

Epoch 1/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5782 - accuracy: 0.7191
Epoch 2/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5637 - accuracy: 0.7250
Epoch 3/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5602 - accuracy: 0.7280
Epoch 4/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5577 - accuracy: 0.7285
Epoch 5/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5574 - accuracy: 0.7286
Epoch 6/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5560 - accuracy: 0.7304
Epoch 7/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5550 - accuracy: 0.7306
Epoch 8/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5542 - accuracy: 0.7309
Epoch 9/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5538 - accuracy: 0.7314
Epoch 10/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5529 - accuracy: 0.7316

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5493 - accuracy: 0.7389 - 177ms/epoch - 662us/step
Loss: 0.5493256449699402, Accuracy: 0.738892138004303


In [21]:
# Export our model to HDF5 file
# model.save("AlphabetSoupCharity_Optimization.keras")
model.save("Models/AlphabetSoupCharity_Optimization.h5")

/Users/peijuwu/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
